In [ ]:
import numpy as np  # 用于数学计算和数组操作
import pandas as pd  # 用于数据处理和分析
import matplotlib.pyplot as plt  # 用于数据可视化
import polars as pl  # 高性能数据处理库，类似于pandas
import datetime  # 用于处理日期和时间
from tqdm import tqdm  # 用于显示进度条
import plotly.express as px  # 用于交互式数据可视化
from plotly.subplots import make_subplots  # 用于创建子图
import plotly.graph_objects as go  # 用于创建更复杂的图表
from metric import score  # 导入自定义的评分函数
from sklearn.model_selection import train_test_split  # 用于划分训练集和测试集

tolerances = {
    'onset': [12, 36, 60, 90, 120, 150, 180, 240, 300, 360],  # 睡眠开始事件的时间容忍度（分钟）
    'wakeup': [12, 36, 60, 90, 120, 150, 180, 240, 300, 360]  # 睡眠结束事件的时间容忍度（分钟）
}

In [ ]:
# 定义时间格式常量
TIME_FORMAT = "%Y-%m-%dT%H:%M:%S%z"  # 这个格式可以处理 -0400, -0500, +0530 等各种时区偏移

# 创建时间转换表达式
timestamp_expr = pl.col('timestamp').str.to_datetime(format=TIME_FORMAT, time_zone='UTC')

dt_transforms = [
    timestamp_expr.alias('timestamp'),  # 转换为UTC时区的datetime
    (timestamp_expr.dt.year() - 2000).cast(pl.UInt8).alias('year'),  # 提取年份（减去2000以节省空间）
    timestamp_expr.dt.month().cast(pl.UInt8).alias('month'),  # 提取月份
    timestamp_expr.dt.day().cast(pl.UInt8).alias('day'),  # 提取日期
    timestamp_expr.dt.hour().cast(pl.UInt8).alias('hour')  # 提取小时
]

data_transforms = [
    pl.col('anglez').cast(pl.Int16), # 将anglez转换为16位整数以节省空间
    (pl.col('enmo')*1000).cast(pl.UInt16), # 将enmo乘以1000并转换为16位无符号整数
]

# 读取训练数据（使用lazy loading提高效率）
train_series = pl.scan_parquet('/home/zhuangzhuohan/sleep_data/train_series.parquet').with_columns(
    dt_transforms + data_transforms
    )

# 读取训练事件数据
train_events = pl.read_csv('/home/zhuangzhuohan/sleep_data/train_events.csv').with_columns(
    dt_transforms
    ).drop_nulls()  # 删除空值

# 读取测试数据（使用lazy loading提高效率）
test_series = pl.scan_parquet('/home/zhuangzhuohan/sleep_data/test_series.parquet').with_columns(
    dt_transforms + data_transforms
    )

# 移除事件数量不匹配的夜晚（确保每个onset对应一个wakeup）
mismatches = train_events.drop_nulls().group_by(['series_id', 'night']).agg([
    ((pl.col('event') == 'onset').sum() == (pl.col('event') == 'wakeup').sum()).alias('balanced')
    ]).sort(by=['series_id', 'night']).filter(~pl.col('balanced'))

for mm in mismatches.to_numpy(): 
    train_events = train_events.filter(~((pl.col('series_id') == mm[0]) & (pl.col('night') == mm[1])))

# 获取唯一的series_id列表
series_ids = train_events['series_id'].unique(maintain_order=True).to_list()

# 更新train_series，只保留有事件数据的series_id
train_series = train_series.filter(pl.col('series_id').is_in(series_ids))

In [ ]:
features, feature_cols = [pl.col('hour')], ['hour']  # 初始化特征列表，先加入小时特征

# 为不同时间窗口创建特征
for mins in [5, 30, 60*2, 60*8] :  # 5分钟、30分钟、2小时、8小时
    
    for var in ['enmo', 'anglez'] :  # 对enmo和anglez两个变量创建特征
        
        # 创建基础统计特征
        features += [
            # 计算滚动平均值（绝对值）
            pl.col(var).rolling_mean(12 * mins, center=True, min_samples=1).abs().cast(pl.UInt16).alias(f'{var}_{mins}m_mean'),
            # 计算滚动最大值（绝对值）
            pl.col(var).rolling_max(12 * mins, center=True, min_samples=1).abs().cast(pl.UInt16).alias(f'{var}_{mins}m_max'),
            # 计算滚动标准差（绝对值）
            pl.col(var).rolling_std(12 * mins, center=True, min_samples=1).abs().cast(pl.UInt16).alias(f'{var}_{mins}m_std')
        ]
        
        # 更新特征列名列表
        feature_cols += [ 
            f'{var}_{mins}m_mean', f'{var}_{mins}m_max', f'{var}_{mins}m_std'
        ]
        
        # 创建一阶差分特征（衡量变化率）
        features += [
            # 计算一阶差分的滚动平均值（绝对值）
            (pl.col(var).diff().abs().rolling_mean(12 * mins, center=True, min_samples=1)*10).abs().cast(pl.UInt32).alias(f'{var}_1v_{mins}m_mean'),
            # 计算一阶差分的滚动最大值（绝对值）
            (pl.col(var).diff().abs().rolling_max(12 * mins, center=True, min_samples=1)*10).abs().cast(pl.UInt32).alias(f'{var}_1v_{mins}m_max'),
            # 计算一阶差分的滚动标准差（绝对值）
            (pl.col(var).diff().abs().rolling_std(12 * mins, center=True, min_samples=1)*10).abs().cast(pl.UInt32).alias(f'{var}_1v_{mins}m_std')
        ]
        
        # 更新特征列名列表
        feature_cols += [ 
            f'{var}_1v_{mins}m_mean', f'{var}_1v_{mins}m_max', f'{var}_1v_{mins}m_std'
        ]

id_cols = ['series_id', 'step', 'timestamp']  # 标识列

# 应用特征变换到训练数据
train_series = train_series.with_columns(
    features
).select(id_cols + feature_cols)  # 只保留需要的列

# 应用特征变换到测试数据
test_series = test_series.with_columns(
    features
).select(id_cols + feature_cols)

In [ ]:
def make_train_dataset(train_data, train_events, drop_nulls=False):
    """
    创建训练数据集的改进版本，修复了一些问题
    
    参数:
    train_data: 训练时间序列数据
    train_events: 训练事件数据
    drop_nulls: 是否删除没有事件记录的日期数据
    
    返回:
    X: 特征矩阵
    y: 标签向量
    """
    
    series_ids = train_data['series_id'].unique(maintain_order=True).to_list()
    X, y = pl.DataFrame(), pl.DataFrame()  # 初始化特征和标签数据框
    
    for idx in tqdm(series_ids):  # 遍历每个series_id
        
        # 标准化样本特征
        sample = train_data.filter(pl.col('series_id') == idx).with_columns(
            [(pl.col(col) / pl.col(col).std()).cast(pl.Float32) for col in feature_cols if col != 'hour']
        )
        
        events = train_events.filter(pl.col('series_id') == idx)  # 获取当前series_id的事件数据
        
        if drop_nulls:
            # 移除没有事件记录的日期的数据点
            sample = sample.filter(
                pl.col('timestamp').dt.date().is_in(events['timestamp'].dt.date())
            )
        
        X = X.vstack(sample[id_cols + feature_cols])  # 添加特征数据
        
        # 修复：使用is_not_null()检查空值
        onsets = events.filter((pl.col('event') == 'onset') & (pl.col('step').is_not_null()))['step'].to_list()
        wakeups = events.filter((pl.col('event') == 'wakeup') & (pl.col('step').is_not_null()))['step'].to_list()
        
        # 修复：使用pl.sum_horizontal替代sum，并添加错误处理
        if onsets and wakeups and len(onsets) == len(wakeups):
            conditions = [(onset <= pl.col('step')) & (pl.col('step') <= wakeup) for onset, wakeup in zip(onsets, wakeups)]
            y = y.vstack(sample.with_columns(
                pl.sum_horizontal(conditions).cast(pl.Boolean).alias('asleep')
            ).select('asleep'))
        else:
            # 如果没有有效的睡眠区间，创建全为False的列
            y = y.vstack(sample.with_columns(
                pl.lit(False).alias('asleep')
            ).select('asleep'))
    
    y = y.to_numpy().ravel()  # 将标签转换为一维数组
    
    return X, y

In [ ]:
def get_events(series, classifier) :
    '''
    将分类器的预测结果转换为睡眠事件（onset和wakeup），并生成提交格式的数据框
    
    参数:
    series: 时间序列数据
    classifier: 训练好的分类器模型
    
    返回:
    events: 包含预测事件的DataFrame，格式符合提交要求
    '''
    
    series_ids = series['series_id'].unique(maintain_order=True).to_list()
    events = pl.DataFrame(schema={'series_id':str, 'step':int, 'event':str, 'score':float})  # 初始化事件数据框

    for idx in tqdm(series_ids) :  # 遍历每个series_id，显示进度条

        # 准备数据并标准化特征
        scale_cols = [col for col in feature_cols if (col != 'hour') & (series[col].std() !=0)]
        X = series.filter(pl.col('series_id') == idx).select(id_cols + feature_cols).with_columns(
            [(pl.col(col) / series[col].std()).cast(pl.Float32) for col in scale_cols]
        )

        # 使用分类器进行预测，获取类别和概率
        preds, probs = classifier.predict(X[feature_cols]), classifier.predict_proba(X[feature_cols])[:, 1]

        # 将预测结果添加到数据框
        X = X.with_columns(
            pl.lit(preds).cast(pl.Int8).alias('prediction'), 
            pl.lit(probs).alias('probability')
                        )
        
        # 检测睡眠开始和结束事件（通过预测值的变化）
        pred_onsets = X.filter(X['prediction'].diff() > 0)['step'].to_list()  # 从0变为1的点为onset
        pred_wakeups = X.filter(X['prediction'].diff() < 0)['step'].to_list()  # 从1变为0的点为wakeup
        
        if len(pred_onsets) > 0 : 
            
            # 确保所有预测的睡眠周期都有开始和结束
            if min(pred_wakeups) < min(pred_onsets) : 
                pred_wakeups = pred_wakeups[1:]  # 移除第一个wakeup（如果它在第一个onset之前）

            if max(pred_onsets) > max(pred_wakeups) :
                pred_onsets = pred_onsets[:-1]  # 移除最后一个onset（如果它在最后一个wakeup之后）

            # 只保留持续时间超过30分钟的睡眠周期
            sleep_periods = [(onset, wakeup) for onset, wakeup in zip(pred_onsets, pred_wakeups) if wakeup - onset >= 6 * 30]

            for onset, wakeup in sleep_periods :
                # 计算睡眠周期内的平均概率作为分数
                score = X.filter((pl.col('step') >= onset) & (pl.col('step') <= wakeup))['probability'].mean()

                # 将睡眠事件添加到数据框
                events = events.vstack(pl.DataFrame().with_columns(
                    pl.Series([idx, idx]).alias('series_id'), 
                    pl.Series([onset, wakeup]).alias('step'),
                    pl.Series(['onset', 'wakeup']).alias('event'),
                    pl.Series([score, score]).alias('score')
                ))

    # 添加行ID列
    events = events.to_pandas().reset_index().rename(columns={'index':'row_id'})

    return events

In [ ]:
# 1. 导入必要的库
from sklearn.model_selection import train_test_split

# 2. 定义列名映射（用于评分函数）
column_names = {
    'series_id_column_name': 'series_id',
    'time_column_name': 'step',
    'event_column_name': 'event',
    'score_column_name': 'score',
}

# 3. 划分训练集和验证集（70%训练，30%验证）
train_ids, val_ids = train_test_split(series_ids, train_size=0.7, random_state=42)

# 4. 收集训练数据，每5分钟取一个数据点（减少数据量）
train_data = train_series.filter(pl.col('series_id').is_in(train_ids)).collect()
# 转换为pandas DataFrame后使用切片方法
train_data = train_data.to_pandas().iloc[::(12 * 5)]  # 每5分钟（12*5步）取一个数据点
train_data = pl.from_pandas(train_data)  # 转回polars DataFrame

# 创建训练事件数据（只包含训练集的事件）
train_solution_series_id = train_events.filter(pl.col('series_id').is_in(train_ids))
train_solution = train_events.filter(pl.col('series_id').is_in(train_ids)).select(['series_id', 'event', 'step']).to_pandas()

# 统计真实的onset和wakeup事件数量
train_solution_onset_count = len(train_solution[train_solution['event'] == 'onset'])
train_solution_wakeup_count = len(train_solution[train_solution['event'] == 'wakeup'])
print(f"train_solution的onset事件数量: {train_solution_onset_count}")
print(f"train_solution的wakeup事件数量: {train_solution_wakeup_count}")

# 5. 创建验证数据
val_data = train_series.filter(pl.col('series_id').is_in(val_ids)).collect()

# 6. 创建验证标签（用于评估模型性能）
val_solution = train_events.filter(pl.col('series_id').is_in(val_ids)).select(['series_id', 'event', 'step']).to_pandas()

# 统计真实的onset和wakeup事件数量
val_solution_onset_count = len(val_solution[val_solution['event'] == 'onset'])
val_solution_wakeup_count = len(val_solution[val_solution['event'] == 'wakeup'])
print(f"val_solution的onset事件数量: {val_solution_onset_count}")
print(f"val_solution的wakeup事件数量: {val_solution_wakeup_count}")

In [ ]:
# 创建训练数据集
X_train, y_train = make_train_dataset(train_data, train_solution_series_id)

In [ ]:
from sklearn.ensemble import RandomForestClassifier

# 初始化随机森林分类器
rf_classifier = RandomForestClassifier(random_state=42)

# 训练分类器（设置超参数）
rf_classifier = RandomForestClassifier(n_estimators=500,  # 500棵树
                                    min_samples_leaf=25,  # 每个叶节点最少25个样本
                                    random_state=42,  # 随机种子，保证结果可重现
                                    n_jobs=-1)  # 使用所有CPU核心

# 拟合模型
rf_classifier.fit(X_train[feature_cols], y_train)

In [ ]:
# 绘制特征重要性图
px.bar(x=feature_cols, 
       y=rf_classifier.feature_importances_,  # 随机森林模型的特征重要性得分
       title='Random forest feature importances'
      )

In [ ]:
# 在验证集上检查模型性能
rf_submission = get_events(val_data, rf_classifier)  # 生成验证集的预测事件

# 统计onset和wakeup事件数量
onset_count = len(rf_submission[rf_submission['event'] == 'onset'])
wakeup_count = len(rf_submission[rf_submission['event'] == 'wakeup'])
print(f"预测的onset事件数量: {onset_count}")
print(f"预测的wakeup事件数量: {wakeup_count}")

# 计算模型得分
print(f"Random forest score: {score(val_solution, rf_submission, tolerances, **column_names)}")

In [ ]:
# 保存分类器到文件
import pickle
with open('rf_classifier_5m_8h.pkl', 'wb') as f:
    pickle.dump(rf_classifier, f)

# 从文件加载分类器（验证保存是否成功）
with open('rf_classifier_5m_8h.pkl', 'rb') as f:
    rf_classifier = pickle.load(f)

In [ ]:
# 释放内存（删除大型变量）
del train_data 

In [ ]:
# 为测试集生成事件预测并保存提交文件
submission = get_events(test_series.collect(), rf_classifier)  # 处理测试数据并生成预测
submission.to_csv('submission.csv', index=False)  # 保存为CSV文件